# Fortran small executable

*Last edited: 2023-12-10*

Some exercises to try to better understand the creation of executables by a compiler. There is also a [Notebook that uses the assembly language](https://efurlanm.github.io/ldi/assembly/small-assembly/).

In [5]:
! gfortran --version

GNU Fortran (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
Copyright (C) 2021 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [39]:
%%writefile tiny.f95
program tiny01
    call exit(42)
end

Overwriting tiny.f95


In [17]:
! wc -c tiny.f95

52 tiny01.f95


In [18]:
! gfortran tiny.f95 ; ./a.out ; echo $?

echo $? will return the exit status of last command

In [20]:
! wc -c a.out

16080 a.out


In [38]:
%%writefile tiny.c
int main(void) {
    return 42;
}

Overwriting tiny.c


In [28]:
! gcc tiny.c ; ./a.out ; echo $?

42


In [23]:
! wc -c a.out

15776 a.out


-s = strip

In [30]:
! gfortran -s tiny.f95 ; wc -c a.out

14400 a.out


In [29]:
! gcc -s tiny.c ; wc -c a.out

14328 a.out


-Os = optimize for size

In [31]:
! gfortran -s -Os tiny.f95 ; wc -c a.out

14400 a.out


In [32]:
! gcc -s -Os tiny.c ; wc -c a.out

14328 a.out


The linker option -nostdlib is used to link a program intended to run standalone. -nostdlib implies the individual options -nodefaultlibs and -nostartfiles

In [35]:
! gfortran -s -nostdlib tiny.f95

/usr/bin/ld: warning: cannot find entry symbol _start; defaulting to 0000000000001040
/usr/bin/ld: /tmp/ccGEs0Cl.o: in function `MAIN__':
tiny01.f95:(.text+0x14): undefined reference to `_gfortran_exit_i4'
/usr/bin/ld: /tmp/ccGEs0Cl.o: in function `main':
tiny01.f95:(.text+0x34): undefined reference to `_gfortran_set_args'
/usr/bin/ld: tiny01.f95:(.text+0x48): undefined reference to `_gfortran_set_options'
collect2: error: ld returned 1 exit status
wc: a.out: No such file or directory


In [36]:
! gcc -s -nostdlib tiny.c ; wc -c a.out

/usr/bin/ld: warning: cannot find entry symbol _start; defaulting to 0000000000001000
13296 a.out


In [41]:
! gcc -s -nostdlib tiny.c ; wc -c a.out

/usr/bin/ld: warning: cannot find entry symbol _start; defaulting to 0000000000001000
13296 a.out


In [42]:
! gcc -s -nostdlib -lgcc tiny.c ; wc -c a.out

/usr/bin/ld: warning: cannot find entry symbol _start; defaulting to 0000000000001000
13296 a.out


The option -nostartfiles instructs the linker to not use the standard system startup functions nor link the code containing those functions.

In [44]:
! gcc -s -nostartfiles tiny.c ; wc -c a.out

/usr/bin/ld: warning: cannot find entry symbol _start; defaulting to 0000000000001000
13296 a.out


In [47]:
! gcc -s -nostartfiles -nostdlib -nodefaultlibs tiny.c ; wc -c a.out

/usr/bin/ld: warning: cannot find entry symbol _start; defaulting to 0000000000001000
13296 a.out


In [48]:
! strip --strip-all a.out ; wc -c a.out

13296 a.out


In [51]:
! strip --strip-all --remove-section=.comment --remove-section=.note* a.out ; wc -c a.out

13016 a.out


In [52]:
! objdump -x a.out


a.out:     file format elf64-x86-64
a.out
architecture: i386:x86-64, flags 0x00000150:
HAS_SYMS, DYNAMIC, D_PAGED
start address 0x0000000000001000

Program Header:
    PHDR off    0x0000000000000040 vaddr 0x0000000000000040 paddr 0x0000000000000040 align 2**3
         filesz 0x00000000000002a0 memsz 0x00000000000002a0 flags r--
  INTERP off    0x0000000000000318 vaddr 0x0000000000000318 paddr 0x0000000000000318 align 2**0
         filesz 0x000000000000001c memsz 0x000000000000001c flags r--
    LOAD off    0x0000000000000000 vaddr 0x0000000000000000 paddr 0x0000000000000000 align 2**12
         filesz 0x00000000000003b9 memsz 0x00000000000003b9 flags r--
    LOAD off    0x0000000000001000 vaddr 0x0000000000001000 paddr 0x0000000000001000 align 2**12
         filesz 0x000000000000000f memsz 0x000000000000000f flags r-x
    LOAD off    0x0000000000002000 vaddr 0x0000000000002000 paddr 0x0000000000002000 align 2**12
         filesz 0x0000000000000050 memsz 0x0000000000000050 flags r--
  

The -Wl,xxx option for gcc passes a comma-separated list of tokens as a space-separated list of arguments to the linker.
To tell the compiler to put functions and data definitions in own sections, we use the little-known -ffunction-sections and -fdata-sections flags to GCC. Then we tell the linker to garbage collect unused sections with --gc-sections.

In [53]:
! gcc -s -Wl,--gc-sections -ffunction-sections -fdata-sections tiny.c ; wc -c a.out

14248 a.out


In [54]:
! objdump -x a.out


a.out:     file format elf64-x86-64
a.out
architecture: i386:x86-64, flags 0x00000150:
HAS_SYMS, DYNAMIC, D_PAGED
start address 0x0000000000001040

Program Header:
    PHDR off    0x0000000000000040 vaddr 0x0000000000000040 paddr 0x0000000000000040 align 2**3
         filesz 0x00000000000002d8 memsz 0x00000000000002d8 flags r--
  INTERP off    0x0000000000000318 vaddr 0x0000000000000318 paddr 0x0000000000000318 align 2**0
         filesz 0x000000000000001c memsz 0x000000000000001c flags r--
    LOAD off    0x0000000000000000 vaddr 0x0000000000000000 paddr 0x0000000000000000 align 2**12
         filesz 0x00000000000005f0 memsz 0x00000000000005f0 flags r--
    LOAD off    0x0000000000001000 vaddr 0x0000000000001000 paddr 0x0000000000001000 align 2**12
         filesz 0x0000000000000145 memsz 0x0000000000000145 flags r-x
    LOAD off    0x0000000000002000 vaddr 0x0000000000002000 paddr 0x0000000000002000 align 2**12
         filesz 0x00000000000000c4 memsz 0x00000000000000c4 flags r--
  

In [56]:
! strip --strip-all --remove-section=.comment --remove-section=.note* --remove-section=.gnu* a.out ; wc -c a.out

13664 a.out


In [57]:
! objdump -x a.out


a.out:     file format elf64-x86-64
a.out
architecture: i386:x86-64, flags 0x00000150:
HAS_SYMS, DYNAMIC, D_PAGED
start address 0x0000000000001040

Program Header:
    PHDR off    0x0000000000000040 vaddr 0x0000000000000040 paddr 0x0000000000000040 align 2**3
         filesz 0x00000000000002a0 memsz 0x00000000000002a0 flags r--
  INTERP off    0x0000000000000318 vaddr 0x0000000000000318 paddr 0x0000000000000318 align 2**0
         filesz 0x000000000000001c memsz 0x000000000000001c flags r--
    LOAD off    0x0000000000000000 vaddr 0x0000000000000000 paddr 0x0000000000000000 align 2**12
         filesz 0x00000000000005f0 memsz 0x00000000000005f0 flags r--
    LOAD off    0x0000000000001000 vaddr 0x0000000000001000 paddr 0x0000000000001000 align 2**12
         filesz 0x0000000000000145 memsz 0x0000000000000145 flags r-x
    LOAD off    0x0000000000002000 vaddr 0x0000000000002000 paddr 0x0000000000002000 align 2**12
         filesz 0x00000000000000c4 memsz 0x00000000000000c4 flags r--
  

In [59]:
! gcc -s tiny.c ; wc -c a.out

14328 a.out


In [58]:
! gcc -s -Wl,--gc-sections tiny.c ; wc -c a.out

14248 a.out


In [61]:
! gcc -s -Wl,--gc-sections -fno-ident -fno-asynchronous-unwind-tables tiny.c ; wc -c a.out

14248 a.out


In [62]:
! gcc -s -nostdlib tiny.c ; wc -c a.out

/usr/bin/ld: warning: cannot find entry symbol _start; defaulting to 0000000000001000
13296 a.out


In [64]:
! gcc -s -nostdlib -nostartfiles tiny.c ; wc -c a.out

/usr/bin/ld: warning: cannot find entry symbol _start; defaulting to 0000000000001000
13296 a.out


In [65]:
! gcc -Os -fdata-sections -ffunction-sections -fipa-pta -Wl,--gc-sections,-O1,--as-needed,--strip-all tiny.c ; wc -c a.out

14248 a.out


In [73]:
! gcc -s -static -nostartfiles tiny.c ;./a.out ; echo $? ; wc -c a.out

/usr/bin/ld: warning: cannot find entry symbol _start; defaulting to 0000000000401000
/bin/bash: line 1: 11653 Segmentation fault      (core dumped) ./a.out
139
8816 a.out


In [76]:
! gcc -s tiny.c ;./a.out ; echo $? ; wc -c a.out

42
14328 a.out


In [80]:
! gcc -s -Os -fdata-sections -ffunction-sections -fipa-pta -Wl,--gc-sections,-O1,--as-needed,--strip-all tiny.c ; ./a.out ; echo $? ; wc -c a.out

42
14248 a.out


In [81]:
! strip --strip-all --remove-section=.comment --remove-section=.note* a.out ; ./a.out ; echo $? ; wc -c a.out

42
13888 a.out


In [82]:
! objdump -x a.out


a.out:     file format elf64-x86-64
a.out
architecture: i386:x86-64, flags 0x00000150:
HAS_SYMS, DYNAMIC, D_PAGED
start address 0x0000000000001050

Program Header:
    PHDR off    0x0000000000000040 vaddr 0x0000000000000040 paddr 0x0000000000000040 align 2**3
         filesz 0x00000000000002a0 memsz 0x00000000000002a0 flags r--
  INTERP off    0x0000000000000318 vaddr 0x0000000000000318 paddr 0x0000000000000318 align 2**0
         filesz 0x000000000000001c memsz 0x000000000000001c flags r--
    LOAD off    0x0000000000000000 vaddr 0x0000000000000000 paddr 0x0000000000000000 align 2**12
         filesz 0x00000000000005f0 memsz 0x00000000000005f0 flags r--
    LOAD off    0x0000000000001000 vaddr 0x0000000000001000 paddr 0x0000000000001000 align 2**12
         filesz 0x0000000000000149 memsz 0x0000000000000149 flags r-x
    LOAD off    0x0000000000002000 vaddr 0x0000000000002000 paddr 0x0000000000002000 align 2**12
         filesz 0x00000000000000b8 memsz 0x00000000000000b8 flags r--
  

In [72]:
! ./a.out ; echo $?

/bin/bash: line 1: 11638 Segmentation fault      (core dumped) ./a.out
139


---

In [100]:
%%writefile tiny.c
#include <unistd.h>
void _start (void) {
  _exit(42);
}

Overwriting tiny.c


In [89]:
! gcc -s -static -nostartfiles tiny.c ; ./a.out ; echo $? ; wc -c a.out

42
8904 a.out


In [90]:
! objdump -x a.out


a.out:     file format elf64-x86-64
a.out
architecture: i386:x86-64, flags 0x00000102:
EXEC_P, D_PAGED
start address 0x0000000000401000

Program Header:
    LOAD off    0x0000000000000000 vaddr 0x0000000000400000 paddr 0x0000000000400000 align 2**12
         filesz 0x0000000000000244 memsz 0x0000000000000244 flags r--
    LOAD off    0x0000000000001000 vaddr 0x0000000000401000 paddr 0x0000000000401000 align 2**12
         filesz 0x0000000000000070 memsz 0x0000000000000070 flags r-x
    LOAD off    0x0000000000002000 vaddr 0x0000000000402000 paddr 0x0000000000402000 align 2**12
         filesz 0x000000000000004c memsz 0x000000000000004c flags r--
    NOTE off    0x0000000000000200 vaddr 0x0000000000400200 paddr 0x0000000000400200 align 2**3
         filesz 0x0000000000000020 memsz 0x0000000000000020 flags r--
    NOTE off    0x0000000000000220 vaddr 0x0000000000400220 paddr 0x0000000000400220 align 2**2
         filesz 0x0000000000000024 memsz 0x0000000000000024 flags r--
     TLS off 

In [91]:
! strip --strip-all --remove-section=.comment --remove-section=.note* a.out ; ./a.out ; echo $? ; wc -c a.out

strip: a.out: warning: empty loadable segment detected at vaddr=0x400000, is this intentional?
42
8624 a.out


In [92]:
! objdump -x a.out


a.out:     file format elf64-x86-64
a.out
architecture: i386:x86-64, flags 0x00000102:
EXEC_P, D_PAGED
start address 0x0000000000401000

Program Header:
    LOAD off    0x0000000000000000 vaddr 0x0000000000400000 paddr 0x0000000000000000 align 2**12
         filesz 0x0000000000000200 memsz 0x0000000000000200 flags r--
    LOAD off    0x0000000000001000 vaddr 0x0000000000401000 paddr 0x0000000000401000 align 2**12
         filesz 0x0000000000000070 memsz 0x0000000000000070 flags r-x
    LOAD off    0x0000000000002000 vaddr 0x0000000000402000 paddr 0x0000000000402000 align 2**12
         filesz 0x000000000000004c memsz 0x000000000000004c flags r--
    NOTE off    0x0000000000000000 vaddr 0x0000000000400200 paddr 0x0000000000000000 align 2**3
         filesz 0x0000000000000000 memsz 0x0000000000000000 flags r--
    NOTE off    0x0000000000000000 vaddr 0x0000000000400220 paddr 0x0000000000000000 align 2**3
         filesz 0x0000000000000000 memsz 0x0000000000000000 flags r--
     TLS off 

In [94]:
! gcc -nostartfiles -Wl,-z,max-page-size=0x1000,-z,norelro tiny.c ; ./a.out ; echo $? ; wc -c a.out

42
10696 a.out


---

In [191]:
%%writefile tiny.c
#include <unistd.h>
#include <sys/syscall.h>
static const char str[] = "Hello world!";
void _start(){
    syscall(SYS_write, 1, str, 12);
    syscall(SYS_exit, 42);
}

Overwriting tiny.c


In [194]:
! gcc -Os -s -static -nostartfiles -Wl,-z,max-page-size=0x1000,-z,norelro tiny.c ; ./a.out ; echo $? ; wc -c a.out

Hello world!42
8984 a.out


In [142]:
! gcc -Os -s -static -nostartfiles -fomit-frame-pointer -fno-exceptions -fno-asynchronous-unwind-tables -Wl,-z,max-page-size=0x1000,-z,norelro tiny.c ; ./a.out ; echo $? ; wc -c a.out

Hello world!42
8960 a.out


In [143]:
! strip --strip-all --remove-section=.comment --remove-section=.note*  --remove-section=.eh_frame* a.out ; ./a.out ; echo $? ; wc -c a.out

strip: a.out: warning: empty loadable segment detected at vaddr=0x400000, is this intentional?
Hello world!42
8560 a.out


In [136]:
! objdump -x a.out


a.out:     file format elf64-x86-64
a.out
architecture: i386:x86-64, flags 0x00000102:
EXEC_P, D_PAGED
start address 0x0000000000401000

Program Header:
    LOAD off    0x0000000000000000 vaddr 0x0000000000400000 paddr 0x0000000000000000 align 2**12
         filesz 0x0000000000000200 memsz 0x0000000000000200 flags r--
    LOAD off    0x0000000000001000 vaddr 0x0000000000401000 paddr 0x0000000000401000 align 2**12
         filesz 0x0000000000000087 memsz 0x0000000000000087 flags r-x
    LOAD off    0x0000000000002000 vaddr 0x0000000000402000 paddr 0x0000000000402000 align 2**12
         filesz 0x000000000000000d memsz 0x000000000000000d flags r--
    NOTE off    0x0000000000000000 vaddr 0x0000000000400200 paddr 0x0000000000000000 align 2**3
         filesz 0x0000000000000000 memsz 0x0000000000000000 flags r--
    NOTE off    0x0000000000000000 vaddr 0x0000000000400220 paddr 0x0000000000000000 align 2**3
         filesz 0x0000000000000000 memsz 0x0000000000000000 flags r--
     TLS off 

In [273]:
%%bash
gcc -Os -s -static -nostartfiles \
-fomit-frame-pointer -fno-exceptions \
-fno-asynchronous-unwind-tables -fno-unwind-tables \
-Wl,-z,norelro \
-Wl,--gc-sections \
tiny.c ; ./a.out ; echo $? ; wc -c a.out

Hello world!42
8960 a.out


In [274]:
! strip --strip-unneeded --strip-all \
--remove-section=.comment --remove-section=.note* --remove-section=.eh_frame* \
a.out ; ./a.out ; echo $? ; wc -c a.out

strip: a.out: warning: empty loadable segment detected at vaddr=0x400000, is this intentional?
Hello world!42
8560 a.out


In [278]:
! objdump -s a.out


a.out:     file format elf64-x86-64

Contents of section .text:
 401000 f30f1efa 50488d15 f40f0000 be010000  ....PH..........
 401010 0031c0bf 01000000 b90c0000 00e81e00  .1..............
 401020 0000be2a 000000bf 3c000000 5a31c0e9  ...*....<...Z1..
 401030 0c000000 662e0f1f 84000000 00006690  ....f.........f.
 401040 f30f1efa 4889f848 89f74889 d64889ca  ....H..H..H..H..
 401050 4d89c24d 89c84c8b 4c24080f 05483d01  M..M..L.L$...H=.
 401060 f0ffff73 01c348c7 c1fcffff fff7d864  ...s..H........d
 401070 89014883 c8ffc3                      ..H....         
Contents of section .rodata:
 402000 48656c6c 6f20776f 726c6421 00        Hello world!.   


In [277]:
! objdump -d a.out


a.out:     file format elf64-x86-64


Disassembly of section .text:

0000000000401000 <.text>:
  401000:	f3 0f 1e fa          	endbr64 
  401004:	50                   	push   %rax
  401005:	48 8d 15 f4 0f 00 00 	lea    0xff4(%rip),%rdx        # 0x402000
  40100c:	be 01 00 00 00       	mov    $0x1,%esi
  401011:	31 c0                	xor    %eax,%eax
  401013:	bf 01 00 00 00       	mov    $0x1,%edi
  401018:	b9 0c 00 00 00       	mov    $0xc,%ecx
  40101d:	e8 1e 00 00 00       	call   0x401040
  401022:	be 2a 00 00 00       	mov    $0x2a,%esi
  401027:	bf 3c 00 00 00       	mov    $0x3c,%edi
  40102c:	5a                   	pop    %rdx
  40102d:	31 c0                	xor    %eax,%eax
  40102f:	e9 0c 00 00 00       	jmp    0x401040
  401034:	66 2e 0f 1f 84 00 00 	cs nopw 0x0(%rax,%rax,1)
  40103b:	00 00 00 
  40103e:	66 90                	xchg   %ax,%ax
  401040:	f3 0f 1e fa          	endbr64 
  401044:	48 89 f8             	mov    %rdi,%rax
  401047:	48 89 f7             	mov    %rsi,%r

---

In [251]:
%%writefile hello.f95
program hello
  print*, 'Hello, World!'
  call exit(42)
end

Overwriting hello.f95


In [252]:
! gfortran hello.f95 ; ./a.out ; echo $? ; wc -c a.out

 Hello, World!
42
16264 a.out


In [279]:
%%bash
gfortran -Os -s -nostartfiles -fomit-frame-pointer -fno-exceptions \
-fno-asynchronous-unwind-tables -fno-unwind-tables \
-Wl,-z,max-page-size=0x1000,-z,norelro hello.f95
./a.out ; echo $? ; wc -c a.out

/usr/bin/ld: warning: cannot find entry symbol _start; defaulting to 0000000000001070


 Hello, World!
42
9976 a.out


In [280]:
%%bash
strip --strip-all \
--remove-section=.comment \
--remove-section=.note* \
--remove-section=.eh_frame* \
--remove-section=.gnu.version \
--remove-section=.gnu.version_r \
--remove-section=.gnu.hash \
a.out
./a.out ; echo $? ; wc -c a.out

./a.out: error while loading shared libraries: ./a.out: unsupported version 0 of Verneed record


127
9480 a.out


In [281]:
! objdump -x a.out


a.out:     file format elf64-x86-64
a.out
architecture: i386:x86-64, flags 0x00000150:
HAS_SYMS, DYNAMIC, D_PAGED
start address 0x0000000000001070

Program Header:
    PHDR off    0x0000000000000040 vaddr 0x0000000000000040 paddr 0x0000000000000040 align 2**3
         filesz 0x00000000000001f8 memsz 0x00000000000001f8 flags r--
  INTERP off    0x0000000000000238 vaddr 0x0000000000000238 paddr 0x0000000000000238 align 2**0
         filesz 0x000000000000001c memsz 0x000000000000001c flags r--
    LOAD off    0x0000000000000000 vaddr 0x0000000000000000 paddr 0x0000000000000000 align 2**12
         filesz 0x00000000000004a8 memsz 0x00000000000004a8 flags r--
    LOAD off    0x0000000000001000 vaddr 0x0000000000001000 paddr 0x0000000000001000 align 2**12
         filesz 0x00000000000000ea memsz 0x00000000000000ea flags r-x
    LOAD off    0x0000000000002000 vaddr 0x0000000000002000 paddr 0x0000000000002000 align 2**12
         filesz 0x000000000000004c memsz 0x000000000000004c flags r--
  

In [242]:
! readelf -S a.out

There are 12 section headers, starting at offset 0x2250:

Section Headers:
  [Nr] Name              Type             Address           Offset
       Size              EntSize          Flags  Link  Info  Align
  [ 0]                   NULL             0000000000000000  00000000
       0000000000000000  0000000000000000           0     0     0
  [ 1] .interp           PROGBITS         0000000000000238  00000238
       000000000000001c  0000000000000000   A       0     0     1
  [ 2] .gnu.hash         GNU_HASH         0000000000000278  00000278
       000000000000001c  0000000000000000   A       3     0     8
  [ 3] .dynsym           DYNSYM           0000000000000298  00000298
       00000000000000a8  0000000000000018   A       4     1     8
  [ 4] .dynstr           STRTAB           0000000000000340  00000340
       00000000000000a6  0000000000000000   A       0     0     1
  [ 5] .rela.plt         RELA             0000000000000418  00000418
       0000000000000090  0000000000000018  AI  

In [268]:
%%bash
gfortran -Os -s \
-fno-asynchronous-unwind-tables \
-Wl,-z,norelro \
-Wl,--gc-sections \
hello.f95
./a.out ; echo $? ; wc -c a.out

 Hello, World!
42
11160 a.out


In [270]:
%%bash
strip --strip-unneeded --strip-all \
--remove-section=.comment \
--remove-section=.note* \
--remove-section=.eh_frame* \
--remove-section=.gnu.version \
a.out
./a.out ; echo $? ; wc -c a.out

 Hello, World!
42
10568 a.out


In [271]:
! objdump -s a.out


a.out:     file format elf64-x86-64

Contents of section .interp:
 02e0 2f6c6962 36342f6c 642d6c69 6e75782d  /lib64/ld-linux-
 02f0 7838362d 36342e73 6f2e3200           x86-64.so.2.    
Contents of section .gnu.hash:
 0368 02000000 0b000000 01000000 06000000  ................
 0378 00008100 00000000 0b000000 00000000  ................
 0388 d165ce6d                             .e.m            
Contents of section .dynsym:
 0390 00000000 00000000 00000000 00000000  ................
 03a0 00000000 00000000 01000000 20000000  ............ ...
 03b0 00000000 00000000 00000000 00000000  ................
 03c0 10000000 20000000 00000000 00000000  .... ...........
 03d0 00000000 00000000 2c000000 20000000  ........,... ...
 03e0 00000000 00000000 00000000 00000000  ................
 03f0 70000000 12000000 00000000 00000000  p...............
 0400 00000000 00000000 58000000 12000000  ........X.......
 0410 00000000 00000000 00000000 00000000  ................
 0420 93000000 12000000 00000000 

In [267]:
! readelf -p .dynstr a.out


String dump of section '.dynstr':
  [     1]  __gmon_start__
  [    10]  _ITM_deregisterTMCloneTable
  [    2c]  _ITM_registerTMCloneTable
  [    46]  _gfortran_exit_i4
  [    58]  _gfortran_st_write_done
  [    70]  _gfortran_transfer_character_write
  [    93]  _gfortran_set_options
  [    a9]  _gfortran_st_write
  [    bc]  _gfortran_set_args
  [    cf]  __cxa_finalize
  [    de]  __libc_start_main
  [    f0]  libgfortran.so.5
  [   101]  libc.so.6
  [   10b]  GLIBC_2.34
  [   116]  GLIBC_2.2.5
  [   122]  GFORTRAN_8



---

In [284]:
%%writefile tiny.c
#include <stdio.h>
int main(void) {
    printf("Hello, world!");
    return 42;
}

Overwriting tiny.c


In [285]:
%%bash
gcc -Os -s \
-fno-asynchronous-unwind-tables \
-Wl,-z,norelro \
-Wl,--gc-sections \
tiny.c
./a.out ; echo $? ; wc -c a.out

Hello, world!42
11152 a.out


In [286]:
%%bash
strip --strip-unneeded --strip-all \
--remove-section=.comment \
--remove-section=.note* \
--remove-section=.eh_frame* \
--remove-section=.gnu.version \
a.out
./a.out ; echo $? ; wc -c a.out

Hello, world!42
10560 a.out


---

In [297]:
%%bash
gcc -s \
-Wl,-z,norelro \
tiny.c
./a.out ; echo $? ; wc -c a.out

Hello, world!42
11208 a.out


In [298]:
%%bash
gfortran -s \
-Wl,-z,norelro \
hello.f95
./a.out ; echo $? ; wc -c a.out

 Hello, World!
42
11248 a.out


In [1]:
%%bash
gfortran -Os -s \
-Wl,-z,norelro \
hello.f95
./a.out ; echo $? ; wc -c a.out

 Hello, World!
42
11240 a.out


In [8]:
%%bash
gfortran -Os -s \
-fno-asynchronous-unwind-tables \
-Wl,-z,norelro \
hello.f95
./a.out ; echo $? ; wc -c a.out

 Hello, World!
42
11168 a.out


In [9]:
%%bash
gfortran -Os -s \
-fno-asynchronous-unwind-tables \
-Wl,-z,norelro \
-Wl,--gc-sections \
hello.f95
./a.out ; echo $? ; wc -c a.out

 Hello, World!
42
11160 a.out


In [15]:
%%bash
gfortran -Os -s -nostartfiles \
-fno-asynchronous-unwind-tables \
-Wl,-z,norelro \
hello.f95
./a.out ; echo $? ; wc -c a.out

/usr/bin/ld: warning: cannot find entry symbol _start; defaulting to 0000000000001070


 Hello, World!
42
9976 a.out


In [18]:
%%bash
gfortran -s -nostartfiles \
-fno-asynchronous-unwind-tables \
-Wl,-z,norelro \
hello.f95
./a.out ; echo $? ; wc -c a.out

/usr/bin/ld: warning: cannot find entry symbol _start; defaulting to 0000000000001070


 Hello, World!
42
9960 a.out


In [20]:
%%bash
gfortran -s -nostartfiles \
-fno-asynchronous-unwind-tables \
-Wl,-z,norelro \
hello.f95
./a.out ; echo $? ; wc -c a.out

/usr/bin/ld: warning: cannot find entry symbol _start; defaulting to 0000000000001070


 Hello, World!
42
9960 a.out


In [26]:
%%bash
gfortran -s -nostartfiles \
-fno-asynchronous-unwind-tables \
-Wl,-z,norelro \
-Wl,--build-id=none \
hello.f95
./a.out ; echo $? ; wc -c a.out

/usr/bin/ld: warning: cannot find entry symbol _start; defaulting to 0000000000001070


 Hello, World!
42
9880 a.out


In [47]:
%%bash
gfortran -s -nostartfiles -no-pie \
-fno-asynchronous-unwind-tables \
-Wl,-z,norelro \
-Wl,--build-id=none \
hello.f95
./a.out ; echo $? ; wc -c a.out

/usr/bin/ld: warning: cannot find entry symbol _start; defaulting to 0000000000401070


 Hello, World!
42
9856 a.out


In [77]:
%%bash
gfortran -s -nostartfiles -no-pie \
-fno-asynchronous-unwind-tables \
-Wl,-z,norelro \
-Wl,--build-id=none \
hello.f95
objcopy -R .comment -R .gnu.version -R .eh_frame -R .gnu.hash a.out
./a.out ; echo $? ; wc -c a.out

/usr/bin/ld: warning: cannot find entry symbol _start; defaulting to 0000000000401070


 Hello, World!
42
9512 a.out


In [78]:
! objdump -x a.out


a.out:     file format elf64-x86-64
a.out
architecture: i386:x86-64, flags 0x00000112:
EXEC_P, HAS_SYMS, D_PAGED
start address 0x0000000000401070

Program Header:
    PHDR off    0x0000000000000040 vaddr 0x0000000000400040 paddr 0x0000000000400040 align 2**3
         filesz 0x00000000000001c0 memsz 0x00000000000001c0 flags r--
  INTERP off    0x0000000000000200 vaddr 0x0000000000400200 paddr 0x0000000000400200 align 2**0
         filesz 0x000000000000001c memsz 0x000000000000001c flags r--
    LOAD off    0x0000000000000000 vaddr 0x0000000000400000 paddr 0x0000000000400000 align 2**12
         filesz 0x0000000000000450 memsz 0x0000000000000450 flags r--
    LOAD off    0x0000000000001000 vaddr 0x0000000000401000 paddr 0x0000000000401000 align 2**12
         filesz 0x0000000000000137 memsz 0x0000000000000137 flags r-x
    LOAD off    0x0000000000002000 vaddr 0x0000000000402000 paddr 0x0000000000402000 align 2**12
         filesz 0x000000000000003c memsz 0x000000000000003c flags r--
   

## References

- <https://fortran-lang.org/en/learn/quickstart/hello_world/>
- <https://wiki.wxwidgets.org/Reducing_Executable_Size>
- <http://www.muppetlabs.com/~breadbox/software/tiny/teensy.html>
- <http://cs107e.github.io/guides/gcc/>
- <http://timelessname.com/elfbin/>
- <https://web.archive.org/web/20100924220817/http://utilitybase.com/article/show/2007/04/09/225/Size+does+matter:+Optimizing+with+size+in+mind+with+GCC>
- <https://stackoverflow.com/questions/33597523/why-is-smallest-compiled-exe-i-can-make-with-gcc-is-67kb>
- <https://stackoverflow.com/questions/67516597/gcc-passing-nostartfiles-to-ld-via-gcc-for-minimal-binary-size>
- <https://stackoverflow.com/questions/65037919/minimal-executable-size-now-10x-larger-after-linking-than-2-years-ago-for-tiny>
- <https://stackoverflow.com/questions/60570279/what-is-a-reasonable-minimum-number-of-assembly-instructions-for-a-small-c-progr>